In [1]:
import os
os.environ['JSBSIM_ROOT_DIR'] = '/home/ubuntu/jsbsim-JSBSim-trusty-v2018a'

import sys
print ('\n'.join(sys.path))


/home/ubuntu/src/cntk/bindings/python
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages
/home/ubuntu/gym-jsbsim
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions
/home/ubuntu/.ipython


In [2]:
import ray
import ray.tune as tune
import gym_jsbsim
import jsbsim
import csv

In [3]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-01-23_11-11-09_3014/logs.
Waiting for redis server at 127.0.0.1:34854 to respond...
Waiting for redis server at 127.0.0.1:13667 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 20.0 GB memory using /dev/shm.

View the web UI at http://localhost:8890/notebooks/ray_ui.ipynb?token=8bd79eb14fa59d4458239427d75232e91c1a5adaaf283a31



{'node_ip_address': None,
 'redis_address': '172.31.4.39:34854',
 'object_store_address': '/tmp/ray/session_2019-01-23_11-11-09_3014/sockets/plasma_store',
 'webui_url': 'http://localhost:8890/notebooks/ray_ui.ipynb?token=8bd79eb14fa59d4458239427d75232e91c1a5adaaf283a31',
 'raylet_socket_name': '/tmp/ray/session_2019-01-23_11-11-09_3014/sockets/raylet'}

In [4]:
def on_episode_start(info):
    print(info.keys())  # -> "env", 'episode"
    episode = info["episode"]
    print("episode {} started".format(episode.episode_id))
    # open a csv file to log states / actions / rewards
    episode.user_data["oa"] = []

def on_episode_step(info):
    episode = info["episode"]
    #print("episode observation: {}".format(episode.last_observation_for()))
    #print("episode observation: {}".format(episode))
    obs = episode.last_observation_for().tolist()
    act = episode.last_action_for().tolist()
    #print("episode.last_observation_for().tolist()", obs)
    #print("episode.last_action_for().tolist())", act)
    oa_list = obs + act
    #print("obs.extend(act)", oa_list)
    #oa_csv = ','.join(str(e) for e in oa_list)
    print(oa_list, type(oa_list))
    #sar_writer.writerow(oa_list)
    #pole_angle = abs(episode.last_observation_for()[2])
    episode.user_data["oa"].append(oa_list)

def on_episode_end(info):
    episode = info["episode"]
    for e in episode.user_data["oa"]:
        print(",".join(str(v) for v in e))
    
    #pole_angle = np.mean(episode.user_data["pole_angles"])
    #print("episode {} ended with length {} and pole angles {}".format(episode.episode_id, episode.length, pole_angle))
    

In [ ]:
#sar_file = open('sar_file.csv', mode='w')
#sar_writer = csv.writer(sar_file, delimiter=',')

tune.run_experiments({
    "my_experiment": {
        "run": "PPO",
        "env": "JSBSim-HeadingControlTask-A320-NoFG-v0",
        "stop": {"episode_reward_mean": -2.8},
        "config": {
            "num_gpus": 1,
            "num_workers": 2,
            "callbacks": {
                "on_episode_start": tune.function(on_episode_start),
                "on_episode_step": tune.function(on_episode_step),
                "on_episode_end": tune.function(on_episode_end)
            },
        },
    },
})

#sar_file.close()

2019-01-23 11:11:13,588	INFO tune.py:134 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run_experiments()
2019-01-23 11:11:13,588	INFO tune.py:144 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/1 GPUs
Memory usage on this node: 3.1/64.4 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 1/1 GPUs
Memory usage on this node: 3.2/64.4 GB
Result logdir: /home/ubuntu/ray_results/my_experiment
RUNNING trials:
 - PPO_JSBSim-HeadingControlTask-A320-NoFG-v0_0:	RUNNING

Result for PPO_JSBSim-HeadingControlTask-A320-NoFG-v0_0:
  custom_metrics: {}
  date: 2019-01-23_11-11-50
  done: false
  episode_len_mean: 67.67241379310344
  episode_reward_max: -3.597784003333744
  episode_reward_mean: -12.018426492808036
  episode_reward_min: -39.22155937671018
  episodes_this_iter: 58
  episodes_total: 58
  experiment_id: 467a37338a2b4c10b066b3c25ac48133
  hostname: ip-172-31-4-39
  info:
    default:
      cur_kl_coeff: 0.19999995827674866
      cur_lr: 4.999999873689376e-05
      entropy: 4.23696756362915
      kl: 0.014178333804011345
      policy_loss: -0.027102947235107422
      total_lo

In [9]:
ray.shutdown()